# Plot neutralization data from collated studies

Import Python modules:

In [ ]:
import os
import re

import altair as alt

import altair_saver

import bindingcalculator

import pandas as pd

import scipy.stats

Read in data from studies and calculate geometric means:

In [ ]:
data = pd.concat([(pd.read_csv(f, na_filter=None)
                   .assign(study=os.path.basename(f).split('_')[0])
                   [['study', 'group', 'sample', 'RBD_mutations', 'fold_change']]
                   )
                  for f in snakemake.input])

geomeans = (
    data
    .groupby(['study', 'group', 'RBD_mutations'])
    .aggregate(n_samples=pd.NamedAgg('sample', 'nunique'),
               fold_change=pd.NamedAgg('fold_change', scipy.stats.gmean),
               )
    .reset_index()
    .assign(sites=lambda x: x['RBD_mutations'].map(lambda s: [int(m[1: -1]) for m in s.split()]),
            group=lambda x: x['group'].str.replace('convalescent', 'infected').str.replace('and', '&'),
            group_first_word=lambda x: x['group'].str.split().str[0],
            study=lambda x: x['study'].map(lambda s: ' & '.join([w for w in re.split('([A-Z][^A-Z]+)', s) if w])),
            study_group=lambda x: x['study'] + ' (' + x['group'] + ')'
            )
    .sort_values(['group_first_word', 'study'])
    )

Create a binding calculator:

In [ ]:
bindcalc = bindingcalculator.BindingCalculator()

geomeans['binding_score'] = geomeans['sites'].map(bindcalc.binding_retained)

geomeans.head()

Plot binding versus fold change:

In [ ]:
chart = (
    alt.Chart(geomeans)
    .encode(x=alt.X('fold_change',
                    title='fold change in neutralization',
                    scale=alt.Scale(type='log',
                                    nice=False,
                                    ),
                    axis=alt.Axis(tickCount=4),
                    ),
            y=alt.Y('binding_score',
                    title='escape score',
                    scale=alt.Scale(type='log',
                                    nice=False,
                                    ),
                    ),
            tooltip=['RBD_mutations',
                     alt.Tooltip('fold_change', format='.2f'),
                     alt.Tooltip('binding_score', format='.2f'),
                     ],
            facet=alt.Facet('study_group',
                            columns=3,
                            title=None,
                            sort=geomeans['study_group'].unique().tolist(),
                            header=alt.Header(labelFontSize=12),
                            ),
            )
    .properties(width=190, height=190)
    .mark_point(filled=True,
                size=100)
    .resolve_scale(x='independent',
                   y='independent')
    )

chart

Save chart:

In [ ]:
html_chart = snakemake.output.html
print(f"Saving to {html_chart}")
chart.save(html_chart)

png_chart = snakemake.output.png
print(f"Saving to {png_chart}")
altair_saver.save(chart, png_chart, vega_cli_options=['-s 4'])